## 引言

量化的本质：通过将模型参数从高精度（例如32位）降低到低精度（例如8位），来缩小模型体积。

本文将采用一种训练后量化方法GPTQ，对前文已经训练并合并过的模型文件进行量化，通过比较模型量化前后的评测指标，来测试量化对模型性能的影响。

GPTQ的核心思想在于：将所有权重压缩到8位或4位量化中，通过最小化与该权重的均方误差来实现。在推理过程中，它将动态地将权重解量化为float16，以提高性能，同时保持较低的内存占用率。

> 注：均方误差是评估两个数值数据集之间差异的一种常用方法，它通过计算量化后权重与原始权重之间的均方误差，并使之最小化，来减少量化过程中引入的误差，以保持模型在推理时的性能。

## 8位量化

#### 加载量化模型

首先引入必要的包，其中：
- auto_gptq: 一个用于模型量化的库，通常用于减少模型的内存占用和计算消耗。
- AutoGPTQForCausalLM: 用于加载和使用经过量化的因果语言模型。
- BaseQuantizeConfig: 定义量化模型时所需的参数，例如量化精度。
- AutoTokenizer：transformers库提供的分词器，用于处理文本分词。

In [10]:
import os
import json
import torch
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig
from transformers import AutoTokenizer

定义量化任务要使用的设备，并指定模型的原始路径`model_path`和量化后的路径`quant_path`。

In [11]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
device = 'cuda'
model_path = "/data2/anti_fraud/models/Qwen2-1__5B-Instruct-anti_fraud_1__0"

In [ ]:
配置量化参数。

In [15]:
quantize_config = BaseQuantizeConfig(
    bits=8, 
    group_size=128,   # 分组量化
    damp_percent=0.01,
    desc_act=False,  
    static_groups=False,
    sym=True,
    true_sequential=True,
    model_name_or_path=None,
    model_file_base_name="model"
)

- group_size：量化时的分组大小，分组量化可以提高计算效率，通常设置为 128 是一个合理的选择，适合大多数模型。
- damp_percent：控制量化过程中对权重的平滑处理，防止过度量化导致的性能下降。默认值 0.01 通常是一个良好的起点，如果量化不佳，可以增加此值。
- desc_act：控制是否使用描述性激活，设置为 False 可以加速推理，如果模型的精度更重要，可以设置为 True。
- static_groups： 是否使用静态分组。静态分组可以提高推理效率， 如果模型结构固定且不需要动态调整，可以设置为 True。否则，保持为 False 以支持动态分组。
- sym： 指定是否使用对称量化。对称量化可以简化计算，如果模型对称性较好，可以设置为 True。
- true_sequential： 控制是否使用真实的顺序量化。真实顺序量化可以提高模型的表现，但可能会增加计算复杂性。如果模型对顺序敏感，可以设置为 True。
- model_file_base_name：指定生成的量化模型文件名称，最终体现在输出文件的命名上。

加载分词器，并根据配置`quantize_config`指定的量化位数来加载模型。

In [16]:
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoGPTQForCausalLM.from_pretrained(model_path, quantize_config)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

#### 校准量化参数

GPTQ采用权重分组量化（如上面的配置中128列为一组），一个分组内的参数采用逐个量化，在每个参数被量化后，需要适当调整这个 block 内其他未量化的参数，以弥补量化造成的精度损失。

![分组量化示意](./img/量化模型/group_quant.png)
因此，GPTQ 量化需要准备校准数据集，我们这里采用一个以前生成的测试数据集作为校准数据。

In [12]:
def load_jsonl(path):
    conversations = []
    with open(path, 'r') as file:
        data = [json.loads(line) for line in file]
        conversations = [dialog['messages'] for dialog in data]
        return conversations

eval_data_path = '/data2/anti_fraud/dataset/test_chatml_0815.jsonl'
conversations = load_jsonl(eval_data_path)

In [ ]:
数据格式是一个标准的聊天模板，示例如下：

In [10]:
conversations[0]

[{'role': 'system', 'content': 'You are a helpful assistant.'},
 {'role': 'user',
  'content': '\n下面是一段对话文本, 请分析对话内容是否有诈骗风险，以json格式输出你的判断结果(is_fraud: true/false)。\n\n\n发言人3: 那就是说看上半年我们的三四月份会不会有一些这个相关的一些这个缓解，就是说这方面的一些矛盾的一些缓解，债务的一个情况的一些缓解，那我们还要继续观察。\n发言人2: 好的，蒋总，那我们看一下那个其他投资者有没有什么其他问题。\n发言人1: 大家好，通过网络端接入的投资者可点击举手连麦等候提问，或在文字交流区提交您的问题，通过电话端接入的投资者请按星一键提问。先按星号键，再按一键，谢谢。大家好，通过网络端接入的投资者可点击举手连麦，然后提问，或在文字交流区提交您的问题。通过电话端接入的投资者请按星一键提问。\n发言人1: 先按星号键，再按数字一键，谢谢。'},
 {'role': 'assistant', 'content': '{"is_fraud": false}'}]

定义一个预处理函数，将文本数据预处理为张量数据。
- tokenizer.apply_chat_template：将消息格式化为Qwen模型需要的提示词格式。
- tokenizer([text])：使用tokenizer对文本进行分词，并将token转换为ID值。
- torch.tensor：将token_id转换为tensor张量。

In [17]:
def preprocess(dataset, max_len=1024):
    data = []
    for msg in dataset:
        text = tokenizer.apply_chat_template(msg, tokenize=False, add_generation_prompt=False)
        model_inputs = tokenizer([text])
        input_ids = torch.tensor(model_inputs.input_ids[:max_len], dtype=torch.int)
        data.append(dict(input_ids=input_ids, attention_mask=input_ids.ne(tokenizer.pad_token_id)))
    return data

dataset = preprocess(conversations)

In [ ]:
配置日志显示格式：

In [ ]:
import logging

logging.basicConfig(
    format="%(asctime)s %(levelname)s [%(name)s] %(message)s", level=logging.INFO, datefmt="%Y-%m-%d %H:%M:%S"
)

开始量化，使用校准数据集来动态调整量化参数，使模型在量化时学习并适应数据分布。

In [18]:
%%time
model.quantize(dataset, cache_examples_on_gpu=False)

INFO - Start quantizing layer 1/28
INFO - Quantizing self_attn.k_proj in layer 1/28...
INFO - Quantizing self_attn.v_proj in layer 1/28...
INFO - Quantizing self_attn.q_proj in layer 1/28...
INFO - Quantizing self_attn.o_proj in layer 1/28...
INFO - Quantizing mlp.up_proj in layer 1/28...
INFO - Quantizing mlp.gate_proj in layer 1/28...
INFO - Quantizing mlp.down_proj in layer 1/28...
INFO - Start quantizing layer 2/28
INFO - Quantizing self_attn.k_proj in layer 2/28...
INFO - Quantizing self_attn.v_proj in layer 2/28...
INFO - Quantizing self_attn.q_proj in layer 2/28...
INFO - Quantizing self_attn.o_proj in layer 2/28...
INFO - Quantizing mlp.up_proj in layer 2/28...
INFO - Quantizing mlp.gate_proj in layer 2/28...
INFO - Quantizing mlp.down_proj in layer 2/28...
INFO - Start quantizing layer 3/28
INFO - Quantizing self_attn.k_proj in layer 3/28...
INFO - Quantizing self_attn.v_proj in layer 3/28...
INFO - Quantizing self_attn.q_proj in layer 3/28...
INFO - Quantizing self_attn.o_pro

CPU times: user 30min 52s, sys: 3min 40s, total: 34min 32s
Wall time: 27min 23s


保存量化后的模型和分词器状态。
> use_safetensors=True 参数表示使用安全张量格式（SafeTensors）进行保存，具有更好的安全性和性能。

In [19]:
quant_path = "/data2/anti_fraud/models/Qwen2-1__5B-Instruct-anti_fraud-gptq-int8"
model.save_quantized(quant_path, use_safetensors=True)
tokenizer.save_pretrained(quant_path)

('/data2/anti_fraud/models/Qwen2-1__5B-Instruct-anti_fraud-gptq-int8/tokenizer_config.json',
 '/data2/anti_fraud/models/Qwen2-1__5B-Instruct-anti_fraud-gptq-int8/special_tokens_map.json',
 '/data2/anti_fraud/models/Qwen2-1__5B-Instruct-anti_fraud-gptq-int8/vocab.json',
 '/data2/anti_fraud/models/Qwen2-1__5B-Instruct-anti_fraud-gptq-int8/merges.txt',
 '/data2/anti_fraud/models/Qwen2-1__5B-Instruct-anti_fraud-gptq-int8/added_tokens.json',
 '/data2/anti_fraud/models/Qwen2-1__5B-Instruct-anti_fraud-gptq-int8/tokenizer.json')

## 4位量化
作为对比，我们也进行一个4位量化，与8位量化的区别只在于量化配置时的参数bits改成了4，其它都不作改变.

In [22]:
quantize_config_int4 = BaseQuantizeConfig(
    bits=4,           # 4位量化
    group_size=128,   # 分组量化
    damp_percent=0.01,
    desc_act=False,  
    static_groups=False,
    sym=True,
    true_sequential=True,
    model_name_or_path=None,
    model_file_base_name="model"
)

In [23]:
model_int4 = AutoGPTQForCausalLM.from_pretrained(model_path, quantize_config_int4)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [24]:
%%time
model_int4.quantize(dataset, cache_examples_on_gpu=False)

INFO - Start quantizing layer 1/28
INFO - Quantizing self_attn.k_proj in layer 1/28...
INFO - Quantizing self_attn.v_proj in layer 1/28...
INFO - Quantizing self_attn.q_proj in layer 1/28...
INFO - Quantizing self_attn.o_proj in layer 1/28...
INFO - Quantizing mlp.up_proj in layer 1/28...
INFO - Quantizing mlp.gate_proj in layer 1/28...
INFO - Quantizing mlp.down_proj in layer 1/28...
INFO - Start quantizing layer 2/28
INFO - Quantizing self_attn.k_proj in layer 2/28...
INFO - Quantizing self_attn.v_proj in layer 2/28...
INFO - Quantizing self_attn.q_proj in layer 2/28...
INFO - Quantizing self_attn.o_proj in layer 2/28...
INFO - Quantizing mlp.up_proj in layer 2/28...
INFO - Quantizing mlp.gate_proj in layer 2/28...
INFO - Quantizing mlp.down_proj in layer 2/28...
INFO - Start quantizing layer 3/28
INFO - Quantizing self_attn.k_proj in layer 3/28...
INFO - Quantizing self_attn.v_proj in layer 3/28...
INFO - Quantizing self_attn.q_proj in layer 3/28...
INFO - Quantizing self_attn.o_pro

CPU times: user 37min 11s, sys: 3min 2s, total: 40min 13s
Wall time: 31min 56s


In [25]:
quant_int4_path = "/data2/anti_fraud/models/Qwen2-1__5B-Instruct-anti_fraud-gptq-int4"
model_int4.save_quantized(quant_int4_path, use_safetensors=True)
tokenizer.save_pretrained(quant_int4_path)

('/data2/anti_fraud/models/Qwen2-1__5B-Instruct-anti_fraud-gptq-int4/tokenizer_config.json',
 '/data2/anti_fraud/models/Qwen2-1__5B-Instruct-anti_fraud-gptq-int4/special_tokens_map.json',
 '/data2/anti_fraud/models/Qwen2-1__5B-Instruct-anti_fraud-gptq-int4/vocab.json',
 '/data2/anti_fraud/models/Qwen2-1__5B-Instruct-anti_fraud-gptq-int4/merges.txt',
 '/data2/anti_fraud/models/Qwen2-1__5B-Instruct-anti_fraud-gptq-int4/added_tokens.json',
 '/data2/anti_fraud/models/Qwen2-1__5B-Instruct-anti_fraud-gptq-int4/tokenizer.json')

## 评测
与[前文](https://golfxiao.blog.csdn.net/article/details/141569237)不同，这里统一采用测试数据集进行评测，以评估模型的最终性能。

#### 原始模型16位评测

In [24]:
%run evaluate.py
testdata_path = '/data2/anti_fraud/dataset/test0819.jsonl'
evaluate(model_path, '', testdata_path, device, batch=True, debug=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

run in batch mode, batch_size=8


progress: 100%|██████████| 2349/2349 [01:52<00:00, 20.87it/s]

tn：1136, fp:31, fn:162, tp:1020
precision: 0.9705042816365367, recall: 0.8629441624365483


#### 量化8位模型评测

In [ ]:
%run evaluate.py
testdata_path = '/data2/anti_fraud/dataset/test0819.jsonl'
model_int8_path = '/data2/anti_fraud/models/Qwen2-1__5B-Instruct-anti_fraud-gptq-int8'
evaluate(model_gptq_path, '', testdata_path, device, batch=True, debug=True)

tn：1134, fp:33, fn:158, tp:1024

precision: 0.9687795648060549, recall: 0.8663282571912013

#### 量化4位模型评测

In [5]:
%run evaluate.py
testdata_path = '/data2/anti_fraud/dataset/test0819.jsonl'
model_int4_path = '/data2/anti_fraud/models/Qwen2-1__5B-Instruct-anti_fraud-gptq-int4'
tokenizer = AutoTokenizer.from_pretrained(model_int4_path)
model_int4_reload = AutoModelForCausalLM.from_pretrained(model_int4_path, device_map=device)
evaluate_with_model(model_int4_reload, tokenizer, testdata_path, device, batch=True, debug=True)

Some weights of the model checkpoint at /data2/anti_fraud/models/Qwen2-1__5B-Instruct-anti_fraud-gptq-int4 were not used when initializing Qwen2ForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.1.self_attn.o_proj.bias', 'model.layers.10.mlp.down_proj.bias', 'model.layers.10.mlp.gate_proj.bias', 'model.layers.10.mlp.up_proj.bias', 'model.layers.10.self_attn.o_proj.bias', 'model.layers.11.mlp.down_proj.bias', 'model.layers.11.mlp.gate_proj.bias', 'model.layers.11.mlp.up_proj.bias', 'model.layers.11.self_attn.o_proj.bias', 'model.layers.12.mlp.down_proj.bias', 'model.layers.12.mlp.gate_proj.bias', 'model.layers.12.mlp.up_proj.bias', 'model.layers.12.self_attn.o_proj.bias', 'model.layers.13.mlp.down_proj.bias', 'model.layers.13.mlp.gate_proj.bias', 'model.la

tn：1081, fp:86, fn:50, tp:1132
precision: 0.9293924466338259, recall: 0.9576988155668359


tn：1081, fp:86, fn:50, tp:1132
precision: 0.9293924466338259, recall: 0.9576988155668359

> 注：4位量化模型这里之所以要单独加载model，是因为GPTQ量化的4位模型有个限制只能在GPU上运行，我们原先的加载方式会报错，具体参考本文最后的`附：4位量化模型加载错误`。

In [ ]:
从这个结果来看，8位量化与原模型差别不大，但4位量化模型与原始模型的性能差别较大，具体体现在：
1. 精确率下降明显，模型在检测欺诈文本时，误报（false positives）数量增加，模型可能会将更多的非欺诈文本错误地分类为欺诈文本。
2. 召回率上升，模型在检测欺诈时漏报（false negatives）的数量减少，这意味着模型在检测欺诈文本时更加激进，尽可能减少漏报，哪怕误报增加。

4位量化比8位量化引入更多的信息丢失和噪声，模型权重和激活值的精度显著下降，最终导致分类效果的变化。

## 模型文件差异

In [28]:
!ls -l /data2/anti_fraud/models/Qwen2-1__5B-Instruct-anti_fraud_1__0

total 3026376
-rw-rw-r-- 1 xiaoguanghua xiaoguanghua         80 Aug 29 11:30 added_tokens.json
-rw-rw-r-- 1 xiaoguanghua xiaoguanghua        748 Aug 29 11:30 config.json
-rw-rw-r-- 1 xiaoguanghua xiaoguanghua        242 Aug 29 11:30 generation_config.json
-rw-rw-r-- 1 xiaoguanghua xiaoguanghua    1671853 Aug 29 11:30 merges.txt
-rw-rw-r-- 1 xiaoguanghua xiaoguanghua 1975314632 Aug 29 11:30 model-00001-of-00002.safetensors
-rw-rw-r-- 1 xiaoguanghua xiaoguanghua 1112152304 Aug 29 11:30 model-00002-of-00002.safetensors
-rw-rw-r-- 1 xiaoguanghua xiaoguanghua      27693 Aug 29 11:30 model.safetensors.index.json
-rw-rw-r-- 1 xiaoguanghua xiaoguanghua        367 Aug 29 11:30 special_tokens_map.json
-rw-rw-r-- 1 xiaoguanghua xiaoguanghua       1532 Aug 29 11:30 tokenizer_config.json
-rw-rw-r-- 1 xiaoguanghua xiaoguanghua    7028043 Aug 29 11:30 tokenizer.json
-rw-rw-r-- 1 xiaoguanghua xiaoguanghua    2776833 Aug 29 11:30 vocab.json


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [7]:
!ls -l /data2/anti_fraud/models/Qwen2-1__5B-Instruct-anti_fraud-gptq-int8

total 2235860
-rw-rw-r-- 1 xiaoguanghua xiaoguanghua         80 Sep 10 11:53 added_tokens.json
-rw-rw-r-- 1 xiaoguanghua xiaoguanghua       1062 Sep 10 11:53 config.json
-rw-rw-r-- 1 xiaoguanghua xiaoguanghua    1671853 Sep 10 11:53 merges.txt
-rw-rw-r-- 1 xiaoguanghua xiaoguanghua 2278014312 Sep 10 11:53 model.safetensors
-rw-rw-r-- 1 xiaoguanghua xiaoguanghua        269 Sep 10 11:53 quantize_config.json
-rw-rw-r-- 1 xiaoguanghua xiaoguanghua        367 Sep 10 11:53 special_tokens_map.json
-rw-rw-r-- 1 xiaoguanghua xiaoguanghua       1532 Sep 10 11:53 tokenizer_config.json
-rw-rw-r-- 1 xiaoguanghua xiaoguanghua    7028043 Sep 10 11:53 tokenizer.json
-rw-rw-r-- 1 xiaoguanghua xiaoguanghua    2776833 Sep 10 11:53 vocab.json


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [6]:
!ls -l /data2/anti_fraud/models/Qwen2-1__5B-Instruct-anti_fraud-gptq-int4

total 1591120
-rw-rw-r-- 1 xiaoguanghua xiaoguanghua         80 Sep 10 12:50 added_tokens.json
-rw-rw-r-- 1 xiaoguanghua xiaoguanghua       1088 Sep 10 18:12 config.json
-rw-rw-r-- 1 xiaoguanghua xiaoguanghua    1671853 Sep 10 12:50 merges.txt
-rw-rw-r-- 1 xiaoguanghua xiaoguanghua 1617798120 Sep 10 12:50 model.safetensors
-rw-rw-r-- 1 xiaoguanghua xiaoguanghua        269 Sep 10 12:50 quantize_config.json
-rw-rw-r-- 1 xiaoguanghua xiaoguanghua        367 Sep 10 12:50 special_tokens_map.json
-rw-rw-r-- 1 xiaoguanghua xiaoguanghua       1532 Sep 10 12:50 tokenizer_config.json
-rw-rw-r-- 1 xiaoguanghua xiaoguanghua    7028043 Sep 10 12:50 tokenizer.json
-rw-rw-r-- 1 xiaoguanghua xiaoguanghua    2776833 Sep 10 12:50 vocab.json


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


可以看到，原始模型、8位量化、4位量化的模型文件大小分别3.08GB、2.27GB、1.61GB，模型文件大小与量化位宽的比例并不完全是线性关系。因为除了模型参数本身之外，还有模型架构、框架开销（pytorch）、优化器的动量和梯度信息等，都会影响着模型文件的总大小。

## 附：4位量化模型加载错误

使用如下代码进行先CPU加载再移到目标GPU时会报`Found modules on cpu/disk`错误：
```
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.bfloat16).eval().to(device)
```
错误详情：
```
ValueError: Found modules on cpu/disk. Using Exllama or Exllamav2 backend requires all the modules to be on GPU.You can deactivate exllama backend by setting `disable_exllama=True` in the quantization config object
```

**原因**：使用GPTQ方式量化int4模型时使用了exllama，这是一种高效的kernel实现，但需要所有模型参数在GPU上，因此对于GPTQ的4位量化模型，先使用CPU加载再移到GPU这种做法行不通。

**解法**：
1. 在模型目录下的config.json文件中，在quantization_config配置块中设置`disable_exllama=true`或者`use_exllama=false`，来禁用exllama，不过可能会影响推理速度。
2. 在加载模型时直接加载到GPU上，类似`from_disk = AutoModelForCausalLM.from_pretrained(path, device_map="cuda:0")`

**小结**：本文通过gptq方法分别对微调后的模型进行了8位量化和4位量化，并对比了量化前后模型的性能指标差异，8位量化模型的性能指标变化小，而4位量化模型的性能指标变异较大。就我们这个场景来说，更适合采用8位量化模型。

## 参考资料
- [大模型量化技术原理](https://mp.weixin.qq.com/s/BErKQW5WzFKbZOrUvknT9A)
- [使用GPTQ、AWQ、BitsAndBytes量化](https://mp.weixin.qq.com/s/ijTcIuLukD0NR27EFjiJ1Q)
- [大模型量化技术前沿](https://mp.weixin.qq.com/s/dZEXUJSioyrk8qL_ykK0mg)
- [哪种量化方法适合你](https://blog.csdn.net/wjjc1017/article/details/136274364)
- [Found modules on cpu/disk错误讨论](https://github.com/QwenLM/Qwen/issues/385)